# Implicit Gyroid structure
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/implicits.ipynb)

In [8]:
import torch
from torchfem import Solid
from torchfem.materials import IsotropicElasticity3D
from torchfem.mesh import cube_hexa
from torchfem.sdfs import gyroid

import pyvista

torch.set_default_dtype(torch.float64)

# Elastic material model
material = IsotropicElasticity3D(E=1000.0, nu=0.3)

# Parameters
scale = 2.0
thickness = 0.1

## Create voxel mesh

In [9]:
nodes, elements = cube_hexa(51, 51, 51, 1.0, 1.0, 1.0)

ecenters = nodes[elements].mean(dim=1)

## Evaluate signed distance function

In [10]:
# Create unstructured mesh
elems = [item for element in elements for item in (len(element), *element)]
cell_types = len(elements) * [pyvista.CellType.HEXAHEDRON]
mesh = pyvista.UnstructuredGrid(elems, cell_types, nodes.tolist())

# Evaluate signed distance function
mesh.point_data["SDF"] = gyroid(scale * nodes).numpy()

## Plot signed distance function

In [11]:
pyvista.set_plot_theme("document")
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
pl.add_mesh(mesh, cmap="coolwarm", show_edges=True, clim=[-1.0, 1.0])
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Plot shell gyroid 

In [12]:
# Contours
contours = mesh.contour([-thickness, thickness])

# Plot
pyvista.set_plot_theme("document")
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
pl.add_mesh(mesh.outline(), color="k")
pl.add_mesh(contours, color="gray")
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [13]:
# Create model
model = Solid(nodes, elements, material)

# Set constraints
model.constraints[nodes[:, 0] == 0.0, 0] = True
model.constraints[nodes[:, 1] == 0.5, 1] = True
model.constraints[nodes[:, 2] == 0.5, 2] = True
model.constraints[nodes[:, 0] == 1.0, 0] = True
model.displacements[nodes[:, 0] == 1.0, 0] = 0.1

# Reduce stiffness outside shell, i.e. in void region
mask = torch.abs(gyroid(scale * ecenters)) > thickness
model.material.C[mask] *= 0.01

## Compute homogenized properties

In [14]:
# Solve
u, f, σ, ε, α = model.solve()

In [16]:
# Evaluate displacement
mesh.point_data["Disp"] = u.numpy()

# Contours
contours = mesh.contour([-thickness, thickness], scalars="SDF")

# Plot
pyvista.set_plot_theme("document")
pl = pyvista.Plotter()
pl.enable_anti_aliasing("ssaa")
pl.add_mesh(mesh.outline(), color="k")
pl.add_mesh(contours, cmap="viridis", scalars="Disp")
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…